# Secrets

A **Secret** stores sensitive values (passwords, tokens, TLS keys) so they can be injected into pods without hardcoding them in images.

Note: Kubernetes Secrets are **base64-encoded**, not encrypted by default. Use RBAC and enable encryption-at-rest.


## Why it is used
- Keep credentials out of images and application code.
- Control access via namespaces/RBAC.
- Mount secrets as files or env vars (prefer files for rotation patterns).


## YAML template (pseudo)
Prefer `stringData` for readability (API will store `data` as base64):

```yaml
apiVersion: v1
kind: Secret
metadata:
  name: db-secret
type: Opaque
stringData:
  DB_USER: "app"
  DB_PASSWORD: "replace-me"
```


## How pods consume a Secret
As env vars:
```yaml
envFrom:
  - secretRef:
      name: db-secret
```

As a mounted file:
```yaml
volumes:
  - name: secrets
    secret:
      secretName: db-secret
containers:
  - name: app
    volumeMounts:
      - name: secrets
        mountPath: /var/run/secrets/app
        readOnly: true
```


## Pitfalls
- Do not commit secrets to git (even if base64).
- Use least-privilege RBAC; many pods should not be able to read many secrets.
- Plan rotation: mounted files can update; env vars usually require restart.

## References
- Secrets: https://kubernetes.io/docs/concepts/configuration/secret/
- Encryption at rest: https://kubernetes.io/docs/tasks/administer-cluster/encrypt-data/
